# GP-VAE Training on Google Colab

This notebook trains the **GP-VAE (Gaussian Process Variational Autoencoder)** model using Google Colab's free GPU.

## What is GP-VAE?
GP-VAE adds a **Gaussian Process prior** to the VAE latent space to model structured correlations:
- **VAE**: Learns image ↔ latent code mapping
- **GP Prior**: Models correlations between latent codes based on:
  - Object identity (same person's face)
  - View angle (front, side, profile)
  - Other factors of variation

## Prerequisites ⚠️
**You MUST have trained VAE weights first!** This model loads pre-trained VAE and fine-tunes it jointly with the GP.

Required files:
- ✅ `out/vae_colab/YYYYMMDD_HHMMSS/vae.cfg.p` - VAE configuration
- ✅ `out/vae_colab/YYYYMMDD_HHMMSS/weights/weights.00000.pt` - Trained VAE weights

## Output Directory Structure:

Each training run creates a **timestamped directory** to avoid overwriting previous runs:
- Format: `./out/gppvae_colab/YYYYMMDD_HHMMSS/`
- Example: `./out/gppvae_colab/20251224_143530/weights/weights.00100.pt`
- This allows you to compare different training runs and keep a history!

Cell 6 below will automatically find your latest VAE training run.

## Setup Instructions:

1. **Open this notebook in VS Code**
2. **Connect to Colab**: Click kernel picker → "Connect to Colab" → Choose **GPU runtime (T4)**
3. **Important**: When prompted with "Alias your server", press Enter
4. **Run cell 2** - it will automatically detect your project location


## 1. Check GPU Availability

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not detected! Go to Runtime → Change runtime type → GPU")

PyTorch version: 2.9.0+cpu
CUDA available: False
⚠️ WARNING: GPU not detected! Go to Runtime → Change runtime type → GPU


## 2. Auto-Detect Project Path

This automatically finds your project files on the Colab runtime.

In [2]:
import os
import sys

# Get current directory
current_dir = os.getcwd()
print(f"📍 Current directory: {current_dir}")

# Check if on Colab and need to mount Drive
if current_dir == '/content':
    print("\n🔄 Mounting Google Drive...")

    try:
        from google.colab import drive
        drive.mount('/content/drive')

        # Check for project in Drive
        drive_path = '/content/drive/MyDrive/gppvae'
        if os.path.exists(drive_path):
            PROJECT_PATH = drive_path
            print(f"✅ Found project in Google Drive: {PROJECT_PATH}")
        else:
            print(f"\n⚠️  Project not found at: {drive_path}")
            print("\nPlease upload your gppvae folder to Google Drive!")
            print("Required structure:")
            print("  MyDrive/gppvae/")
            print("    ├── GPPVAE/")
            print("    ├── data/faceplace/data_faces.h5")
            print("    └── out/vae_colab/YYYYMMDD_HHMMSS/")
            print("        ├── vae.cfg.p")
            print("        └── weights/weights.00000.pt")
            PROJECT_PATH = '/content'
    except Exception as e:
        print(f"Could not mount Drive: {e}")
        PROJECT_PATH = '/content'
else:
    # Running via VS Code sync
    if 'notebooks' in current_dir:
        PROJECT_PATH = os.path.dirname(current_dir)
    else:
        PROJECT_PATH = current_dir
    print(f"💻 Using project path: {PROJECT_PATH}")

# Verify structure
print(f"\n📁 Contents of {PROJECT_PATH}:")
if os.path.exists(PROJECT_PATH):
    items = os.listdir(PROJECT_PATH)
    for item in sorted(items)[:15]:
        item_path = os.path.join(PROJECT_PATH, item)
        if os.path.isdir(item_path):
            print(f"   📂 {item}/")
        else:
            print(f"   📄 {item}")

    # Check required files (with timestamped directory structure)
    print(f"\n🔍 Checking required files:")
    required = {
        'GPPVAE code': os.path.exists(os.path.join(PROJECT_PATH, 'GPPVAE')),
        'data/faceplace': os.path.exists(os.path.join(PROJECT_PATH, 'data/faceplace')),
        'data_faces.h5': os.path.exists(os.path.join(PROJECT_PATH, 'data/faceplace/data_faces.h5')),
    }

    # Check for VAE runs (timestamped subdirectories)
    vae_base_dir = os.path.join(PROJECT_PATH, 'out/vae_colab')
    vae_run_found = False
    vae_weights_found = False

    if os.path.exists(vae_base_dir):
        # Look for timestamped subdirectories
        potential_runs = [d for d in os.listdir(vae_base_dir)
                         if os.path.isdir(os.path.join(vae_base_dir, d)) and d[0].isdigit()]

        for run_dir in potential_runs:
            run_path = os.path.join(vae_base_dir, run_dir)
            cfg_path = os.path.join(run_path, 'vae.cfg.p')
            weights_dir = os.path.join(run_path, 'weights')

            if os.path.exists(cfg_path):
                vae_run_found = True

            if os.path.exists(weights_dir):
                weight_files = [f for f in os.listdir(weights_dir) if f.endswith('.pt')]
                if weight_files:
                    vae_weights_found = True
                    break

    required['VAE config'] = vae_run_found
    required['VAE weights'] = vae_weights_found

    for name, exists in required.items():
        status = "✅" if exists else "❌"
        print(f"   {status} {name}")

    # Show VAE runs if they exist
    if os.path.exists(vae_base_dir):
        potential_runs = sorted([d for d in os.listdir(vae_base_dir)
                                if os.path.isdir(os.path.join(vae_base_dir, d)) and d[0].isdigit()],
                               reverse=True)

        if potential_runs:
            print(f"\n📦 Found {len(potential_runs)} VAE training run(s):")
            for i, run_dir in enumerate(potential_runs[:3], 1):  # Show latest 3
                run_path = os.path.join(vae_base_dir, run_dir)
                weights_dir = os.path.join(run_path, 'weights')

                if os.path.exists(weights_dir):
                    weight_files = sorted([f for f in os.listdir(weights_dir) if f.endswith('.pt')])
                    print(f"   {i}. {run_dir}/ ({len(weight_files)} checkpoints)")
                    if weight_files:
                        print(f"      Latest: {weight_files[-1]}")

            if len(potential_runs) > 3:
                print(f"   ... and {len(potential_runs) - 3} more")

            print(f"\n💡 Cell 6 below will help you choose which run to use")

    if not all(required.values()):
        print(f"\n⚠️  Missing required files!")
        if not required['VAE weights']:
            print("\n🚨 CRITICAL: No trained VAE weights found!")
            print("   You must train VAE first before running GP-VAE")
            print("   Use the train_vae_colab.ipynb notebook")
else:
    print(f"❌ Path doesn't exist: {PROJECT_PATH}")


📍 Current directory: /content

🔄 Mounting Google Drive...
Mounted at /content/drive
✅ Found project in Google Drive: /content/drive/MyDrive/gppvae

📁 Contents of /content/drive/MyDrive/gppvae:
   📂 GPPVAE/
   📂 data/
   📄 environment.yml
   📂 notebooks/
   📂 out/

🔍 Checking required files:
   ✅ GPPVAE code
   ✅ data/faceplace
   ✅ data_faces.h5
   ✅ VAE config
   ✅ VAE weights

📦 Found 3 VAE training run(s):
   1. 20251224_171841/ (11 checkpoints)
      Latest: weights.00099.pt
   2. 20251224_171753/ (0 checkpoints)
   3. 20251224_120136/ (16 checkpoints)
      Latest: weights.00140.pt

💡 Cell 6 below will help you choose which run to use


## 3. Install Dependencies

In [3]:
# Install required packages
!pip install -q wandb==0.12.21 imageio==2.15.0 pyyaml

# Verify installations
import wandb
import imageio
import yaml
import numpy as np
print("✅ All dependencies installed successfully!")

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
✅ All dependencies installed successfully!


## 4. Login to Weights & Biases (Optional)

Track your experiments with W&B for better monitoring.

In [4]:
import wandb
wandb.login()

# Or run offline without W&B:
# import os
# os.environ['WANDB_MODE'] = 'offline'

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: minh1008 (minh1008-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## 5. Navigate to Project Directory

In [5]:
import os
import sys

os.chdir(PROJECT_PATH)
print(f"Current directory: {os.getcwd()}")

# Add to Python path
sys.path.insert(0, os.path.join(PROJECT_PATH, 'GPPVAE/pysrc/faceplace'))

print("\nProject structure:")
!ls -la

Current directory: /content/drive/MyDrive/gppvae

Project structure:
total 17
drwx------ 3 root root 4096 Dec 23 14:09 data
-rw------- 1 root root  258 Dec 23 11:40 environment.yml
drwx------ 2 root root 4096 Dec 23 14:09 GPPVAE
drwx------ 2 root root 4096 Dec 23 14:09 notebooks
drwx------ 3 root root 4096 Dec 23 14:21 out


## 6. Verify VAE Weights

**Critical check:** Make sure you have trained VAE weights!

In [6]:
import os
import pickle
import glob

# Check for VAE runs (may be in timestamped subdirectories)
vae_base_dir = './out/vae_colab'
vae_runs = []

if os.path.exists(vae_base_dir):
    # Look for timestamped subdirectories
    potential_runs = [d for d in os.listdir(vae_base_dir) if os.path.isdir(os.path.join(vae_base_dir, d))]
    for run_dir in sorted(potential_runs, reverse=True):  # Most recent first
        run_path = os.path.join(vae_base_dir, run_dir)
        cfg_path = os.path.join(run_path, 'vae.cfg.p')
        weights_dir = os.path.join(run_path, 'weights')

        if os.path.exists(cfg_path) and os.path.exists(weights_dir):
            weight_files = sorted([f for f in os.listdir(weights_dir) if f.endswith('.pt')])
            if weight_files:
                vae_runs.append({
                    'run_dir': run_dir,
                    'cfg_path': cfg_path,
                    'weights_dir': weights_dir,
                    'weight_files': weight_files
                })

if vae_runs:
    print(f"✅ Found {len(vae_runs)} VAE training run(s):\n")

    for i, run in enumerate(vae_runs, 1):
        print(f"Run {i}: {run['run_dir']}")

        # Load and show config
        vae_cfg = pickle.load(open(run['cfg_path'], 'rb'))
        print(f"   Config: zdim={vae_cfg.get('zdim', 'N/A')}, nf={vae_cfg.get('nf', 'N/A')}")

        # Show checkpoints
        print(f"   Checkpoints: {len(run['weight_files'])} files")
        if len(run['weight_files']) <= 3:
            for wf in run['weight_files']:
                print(f"      📦 {wf}")
        else:
            print(f"      📦 {run['weight_files'][0]} ... {run['weight_files'][-1]}")
        print()

    # Recommendation
    latest_run = vae_runs[0]
    latest_weight = latest_run['weight_files'][-1]
    recommended_path = os.path.join(latest_run['weights_dir'], latest_weight)

    print(f"💡 Recommendation:")
    print(f"   Use latest run: {latest_run['run_dir']}")
    print(f"   Latest checkpoint: {latest_weight}")
    print(f"   \n   Set in next cell:")
    print(f"   CONFIG['vae_cfg'] = '{latest_run['cfg_path']}'")
    print(f"   CONFIG['vae_weights'] = '{recommended_path}'")

else:
    print("❌ No trained VAE runs found!")
    print("\n   Please train VAE first using train_vae_colab.ipynb")
    print(f"   Expected location: {vae_base_dir}/YYYYMMDD_HHMMSS/")


✅ Found 2 VAE training run(s):

Run 1: 20251224_171841
   Config: zdim=256, nf=32
   Checkpoints: 11 files
      📦 weights.00000.pt ... weights.00099.pt

Run 2: 20251224_120136
   Config: zdim=256, nf=32
   Checkpoints: 16 files
      📦 weights.00000.pt ... weights.00140.pt

💡 Recommendation:
   Use latest run: 20251224_171841
   Latest checkpoint: weights.00099.pt
   
   Set in next cell:
   CONFIG['vae_cfg'] = './out/vae_colab/20251224_171841/vae.cfg.p'
   CONFIG['vae_weights'] = './out/vae_colab/20251224_171841/weights/weights.00099.pt'


## 8. Configure GP-VAE Training

Adjust these parameters as needed:

## 7. Choose View Kernel 🔬

**NEW: Kernel Selection for View Correlations**

The view kernel models how correlations between face angles (0°, 15°, 30°, ..., 90°) are structured.

### Available Kernels:

1. **`'legacy'`** - Original implementation (normalized embeddings, 81 params)
   - Most flexible but can overfit
   - Good baseline for comparison

2. **`'fullrank'`** - Direct full-rank covariance (45 params)
   - Flexible but still many parameters
   - Better than legacy due to fewer constraints

3. **`'periodic'`** ⭐ **RECOMMENDED** - Periodic kernel (1 param: lengthscale)
   - Knows that 0° = 360° (periodicity!)
   - Smooth correlations between nearby angles
   - Massive regularization (only 1 parameter)
   - Best for rotation data

4. **`'vonmises'`** ⭐ **RECOMMENDED** - Von Mises kernel (1 param: kappa)
   - Designed specifically for circular/angular data
   - Similar to Periodic but different parameterization
   - Also best for rotation data

5. **`'matern'`** - Matérn kernel (1 param: lengthscale)
   - More realistic than RBF, less smooth
   - Good for modeling realistic correlations
   - Can choose smoothness: nu=1.5 or nu=2.5

6. **`'linear'`** - Low-rank linear (rank×9 params)
   - Original GP-VAE kernel from Casale et al. (2018)
   - Good middle-ground

7. **`'rbf'`** - RBF/Gaussian (1 param: lengthscale)
   - Smooth but NOT periodic
   - Use only if views don't wrap around

### Expected Performance:

| Metric | Legacy | FullRank | Periodic | VonMises | Matérn |
|--------|--------|----------|----------|----------|--------|
| Val MSE | Medium | Medium | **Best** | **Best** | Good |
| Out-of-sample | Worst | Bad | **Best** | **Best** | Good |
| Overfitting | High | Medium | Low | Low | Low |
| Parameters | 81 | 45 | 1 | 1 | 1 |
| Smoothness | - | - | Very smooth | Very smooth | Adjustable |

**Recommendation**:
- **Best for rotations**: `'periodic'` or `'vonmises'`
- **More realistic**: `'matern'` (less smooth than periodic)

In [ ]:
# ============================================================================
# VIEW SPLIT CONFIGURATION - For Hard Held-Out Views Experiment
# ============================================================================

# Experiment mode
VIEW_SPLIT_MODE = 'by_view'  # 'random' or 'by_view'

# View angle mapping (after angular ordering fix):
# Index 0: 90L (-90°), 1: 60L (-60°), 2: 45L (-45°), 3: 30L (-30°), 4: 00F (0°),
# Index 5: 30R (+30°), 6: 45R (+45°), 7: 60R (+60°), 8: 90R (+90°)

if VIEW_SPLIT_MODE == 'by_view':
    # EXPERIMENT 1: Train on central views, test on extreme angles
    TRAIN_VIEW_INDICES = [3, 4, 5]  # -30L, 00F, 30R (60° range)
    VAL_VIEW_INDICES = [0, 1, 2, 6, 7, 8]  # Extreme angles (±45°, ±60°, ±90°)
    
    print("🔬 EXPERIMENT MODE: Hard Held-Out Views")
    print("=" * 60)
    print("Training views (central):")
    print("  Index 3: 30L (-30°)")
    print("  Index 4: 00F (  0°)")
    print("  Index 5: 30R (+30°)")
    print("\nValidation views (extreme):")
    print("  Index 0: 90L (-90°)")
    print("  Index 1: 60L (-60°)")
    print("  Index 2: 45L (-45°)")
    print("  Index 6: 45R (+45°)")
    print("  Index 7: 60R (+60°)")
    print("  Index 8: 90R (+90°)")
    print("=" * 60)
else:
    TRAIN_VIEW_INDICES = None
    VAL_VIEW_INDICES = None
    print("📊 Standard Mode: Random 90/10 train/val split")

In [7]:
# ============================================================================
# KERNEL CONFIGURATION - Choose one option below
# ============================================================================

# Option 1: Periodic kernel (RECOMMENDED for face rotations)
# KERNEL_CONFIG = {
#     'view_kernel': 'periodic',
#     'kernel_kwargs': {'lengthscale': 1.0}
# }

# Option 2: Von Mises kernel (RECOMMENDED alternative)
# KERNEL_CONFIG = {
#     'view_kernel': 'vonmises',
#     'kernel_kwargs': {'kappa': 1.0}
# }

# Option 3: Matérn kernel (realistic, less smooth than periodic)
# KERNEL_CONFIG = {
#     'view_kernel': 'matern',
#     'kernel_kwargs': {'lengthscale': 1.0, 'nu': 1.5}  # nu=1.5 or nu=2.5
# }

# Option 4: Legacy (original implementation - baseline)
KERNEL_CONFIG = {
    'view_kernel': 'legacy',
    'kernel_kwargs': {}
}

# Option 5: Full Rank (flexible, 45 params)
# KERNEL_CONFIG = {
#     'view_kernel': 'fullrank',
#     'kernel_kwargs': {}
# }

# Option 6: Linear low-rank (original GP-VAE paper)
# KERNEL_CONFIG = {
#     'view_kernel': 'linear',
#     'kernel_kwargs': {'rank': 3}
# }

# Option 7: RBF (smooth but not periodic)
# KERNEL_CONFIG = {
#     'view_kernel': 'rbf',
#     'kernel_kwargs': {'lengthscale': 1.0}
# }

print("Selected Kernel Configuration:")
print("=" * 60)
print(f"Kernel type: {KERNEL_CONFIG['view_kernel']}")
if KERNEL_CONFIG['kernel_kwargs']:
    print(f"Parameters: {KERNEL_CONFIG['kernel_kwargs']}")
else:
    print("Parameters: (default)")
print("=" * 60)

Selected Kernel Configuration:
Kernel type: legacy
Parameters: (default)


In [ ]:
from datetime import datetime

# GP-VAE Training configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
kernel_name = KERNEL_CONFIG['view_kernel']

# Include view split mode in directory name
view_mode_str = 'central_views' if VIEW_SPLIT_MODE == 'by_view' else 'random'

CONFIG = {
    'data': './data/faceplace/data_faces.h5',
    # Output directory now includes kernel name AND view split mode
    'outdir': f'./out/gppvae_colab/{kernel_name}_{view_mode_str}_{timestamp}',
    'vae_cfg': './out/vae_colab/20251224_171841/vae.cfg.p',
    'vae_weights': './out/vae_colab/20251224_171841/weights/weights.00099.pt',

    # Training hyperparameters
    'epochs': 100,
    'batch_size': 64,
    'vae_lr': 0.001,
    'gp_lr': 0.001,
    'xdim': 64,

    # Kernel configuration
    'view_kernel': KERNEL_CONFIG['view_kernel'],
    'kernel_kwargs': KERNEL_CONFIG['kernel_kwargs'],

    # Experiment configuration (NEW)
    'view_split_mode': VIEW_SPLIT_MODE,
    'train_view_indices': TRAIN_VIEW_INDICES,
    'val_view_indices': VAL_VIEW_INDICES,

    # Logging
    'epoch_cb': 10,
    'use_wandb': True,
    'wandb_project': 'gppvae-exp1',
    'wandb_run_name': f'exp1_{kernel_name}_{view_mode_str}_{timestamp}',
    'seed': 0,
}

print("GP-VAE Training Configuration:")
print("=" * 60)
for key, value in CONFIG.items():
    if key in ['train_view_indices', 'val_view_indices'] and value is not None:
        print(f"  {key:20s}: {value}")
    elif key not in ['train_view_indices', 'val_view_indices']:
        print(f"  {key:20s}: {value}")
print("=" * 60)

# Verify VAE weights path
if not os.path.exists(CONFIG['vae_weights']):
    print(f"\n⚠️  WARNING: VAE weights not found at:")
    print(f"   {CONFIG['vae_weights']}")

print(f"\n✅ Output will be saved to:")
print(f"   {CONFIG['outdir']}")
print(f"\n   Directory name includes kernel type AND experiment mode!")

GP-VAE Training Configuration:
  data                : ./data/faceplace/data_faces.h5
  outdir              : ./out/gppvae_colab/legacy_20251224_173025
  vae_cfg             : ./out/vae_colab/20251224_171841/vae.cfg.p
  vae_weights         : ./out/vae_colab/20251224_171841/weights/weights.00099.pt
  epochs              : 100
  batch_size          : 64
  vae_lr              : 0.001
  gp_lr               : 0.001
  xdim                : 64
  view_kernel         : legacy
  kernel_kwargs       : {}
  epoch_cb            : 10
  use_wandb           : True
  wandb_project       : gppvae
  wandb_run_name      : colab_kernel_legacy_20251224_173025
  seed                : 0

✅ Output will be saved to:
   ./out/gppvae_colab/legacy_20251224_173025

   Directory name includes kernel type for easy comparison!


## 9. Import Training Modules

In [ ]:
os.chdir(os.path.join(PROJECT_PATH, 'GPPVAE/pysrc/faceplace'))

# Import modules
import matplotlib
matplotlib.use('Agg')

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from vae import FaceVAE
from vmod import Vmodel
from gp import GP
import h5py
import numpy as np
import logging
import pylab as pl
from utils import smartSum, smartAppendDict, smartAppend, export_scripts
from callbacks import callback_gppvae
import pickle
import time
import wandb

# IMPORTANT: Use experiment 1 data parser with view-based splitting
from data_parser_exp1 import read_face_data, FaceDataset

print("✅ All modules imported successfully!")
print("✅ Using data_parser_exp1 for view-based splitting")

✅ All modules imported successfully!


## 10. Setup Training Environment

In [10]:
# Go back to project root
os.chdir(PROJECT_PATH)

# Create output directories
outdir = CONFIG['outdir']
wdir = os.path.join(outdir, "weights")
fdir = os.path.join(outdir, "plots")
os.makedirs(wdir, exist_ok=True)
os.makedirs(fdir, exist_ok=True)

# Setup device (GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Setup logging
log_format = "%(asctime)s %(message)s"
logging.basicConfig(
    level=logging.INFO,
    format=log_format,
    datefmt="%m/%d %I:%M:%S %p",
)
fh = logging.FileHandler(os.path.join(outdir, "log.txt"))
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)

# Copy code to output
export_scripts(os.path.join(outdir, "scripts"))

print("✅ Training environment setup complete!")
print(f"   Outputs will be saved to: {outdir}")

Using device: cpu
✅ Training environment setup complete!
   Outputs will be saved to: ./out/gppvae_colab/legacy_20251224_173025


## 10. Initialize Models and Data

This cell:
1. Loads pre-trained VAE
2. Creates GP and Vmodel
3. Loads dataset
4. Sets up optimizers

In [ ]:
# Set random seed
torch.manual_seed(CONFIG['seed'])

# Initialize W&B
if CONFIG['use_wandb']:
    wandb.init(
        project=CONFIG['wandb_project'],
        name=CONFIG['wandb_run_name'],
        config=CONFIG
    )

# Load VAE configuration
vae_cfg = pickle.load(open(CONFIG['vae_cfg'], "rb"))
print(f"VAE config: {vae_cfg}")

# Load pre-trained VAE
print("\nLoading pre-trained VAE...")
vae = FaceVAE(**vae_cfg).to(device)
vae_state = torch.load(CONFIG['vae_weights'], map_location=device)
vae.load_state_dict(vae_state)
print(f"✅ VAE loaded from {CONFIG['vae_weights']}")
print(f"   Total VAE parameters: {sum(p.numel() for p in vae.parameters()):,}")

# Load data with experiment configuration
print("\nLoading dataset...")
img, obj, view = read_face_data(
    CONFIG['data'],
    view_split_mode=CONFIG['view_split_mode'],
    train_view_indices=CONFIG.get('train_view_indices'),
    val_view_indices=CONFIG.get('val_view_indices')
)

train_data = FaceDataset(img["train"], obj["train"], view["train"])
val_data = FaceDataset(img["val"], obj["val"], view["val"])
train_queue = DataLoader(train_data, batch_size=CONFIG['batch_size'], shuffle=True)
val_queue = DataLoader(val_data, batch_size=CONFIG['batch_size'], shuffle=False)

# Enhanced diagnostic logging
print(f"\n✅ Data loaded:")
print(f"   Training samples: {len(train_data)}")
print(f"   Validation samples: {len(val_data)}")
print(f"   Unique train views: {np.unique(view['train'].numpy())}")
print(f"   Unique val views: {np.unique(view['val'].numpy())}")
print(f"   Unique train identities: {len(np.unique(obj['train'].numpy()))}")
print(f"   Unique val identities: {len(np.unique(obj['val'].numpy()))}")

# Validation checks for experiment mode
if CONFIG['view_split_mode'] == 'by_view':
    print("\n🔍 Experiment Mode Validation Checks:")
    
    # Check 1: View split correctness
    train_views_set = set(np.unique(view['train'].numpy().flatten()).astype(int))
    val_views_set = set(np.unique(view['val'].numpy().flatten()).astype(int))
    
    assert train_views_set == set(CONFIG['train_view_indices']), f"Train views mismatch!"
    assert val_views_set == set(CONFIG['val_view_indices']), f"Val views mismatch!"
    assert len(train_views_set & val_views_set) == 0, "Train and val views overlap!"
    print("   ✅ View split verified correctly!")
    
    # Check 2: Identity coverage
    train_ids = set(np.unique(obj['train'].numpy()))
    val_ids = set(np.unique(obj['val'].numpy()))
    assert train_ids == val_ids, "Identity sets don't match between train/val!"
    print(f"   ✅ All {len(train_ids)} identities present in both train/val!")
    
    # Check 3: Sample distribution
    train_samples_per_id = len(img['train']) / len(train_ids)
    val_samples_per_id = len(img['val']) / len(val_ids)
    print(f"   ✅ Train samples per identity: {train_samples_per_id:.1f} (expected: {len(CONFIG['train_view_indices'])}.0)")
    print(f"   ✅ Val samples per identity: {val_samples_per_id:.1f} (expected: {len(CONFIG['val_view_indices'])}.0)")

# Create object and view variables for GP
Dt = Variable(obj["train"][:, 0].long(), requires_grad=False).cuda()
Wt = Variable(view["train"][:, 0].long(), requires_grad=False).cuda()
Dv = Variable(obj["val"][:, 0].long(), requires_grad=False).cuda()
Wv = Variable(view["val"][:, 0].long(), requires_grad=False).cuda()

# Initialize GP and Vmodel
print("\nInitializing GP-VAE components...")

# IMPORTANT: Count unique identities and views AFTER filtering
# The data parser may have filtered out some identities
train_identities = np.unique(obj["train"].numpy())
train_views = np.unique(view["train"].numpy())
val_identities = np.unique(obj["val"].numpy())
val_views = np.unique(view["val"].numpy())

# For Vmodel, we need the TOTAL number of unique objects and views across ALL data
# (not just train, since we'll use it for val too)
all_identities = np.unique(np.concatenate([obj["train"].numpy(), obj["val"].numpy()]))
all_views = np.unique(np.concatenate([view["train"].numpy(), view["val"].numpy()]))

P = len(all_identities)  # Number of unique objects (people)
Q = len(all_views)  # Number of unique views (angles)

print(f"   Objects (people): {P}")
print(f"   Views (angles): {Q}")
print(f"   Train identities: {len(train_identities)}, Val identities: {len(val_identities)}")
print(f"   Train views: {sorted(train_views.tolist())}, Val views: {sorted(val_views.tolist())}")

vm = Vmodel(
    P, Q,
    p=CONFIG['xdim'],
    q=Q,
    view_kernel=CONFIG['view_kernel'],
    **CONFIG['kernel_kwargs']
).cuda()

print(f"\n🔬 Initializing view kernel: '{CONFIG['view_kernel']}'")
if CONFIG['kernel_kwargs']:
    print(f"   Kernel parameters: {CONFIG['kernel_kwargs']}")
else:
    print(f"   Kernel parameters: (default)")

gp = GP(n_rand_effs=1).to(device)

# Combine GP parameters (Vmodel + GP)
gp_params = nn.ParameterList()
gp_params.extend(vm.parameters())
gp_params.extend(gp.parameters())

print(f"✅ GP-VAE components initialized:")
print(f"   Vmodel parameters: {sum(p.numel() for p in vm.parameters()):,}")
print(f"   GP parameters: {sum(p.numel() for p in gp.parameters()):,}")
print(f"   Total trainable: {sum(p.numel() for p in vae.parameters()) + sum(p.numel() for p in gp_params):,}")

# Create optimizers (separate for VAE and GP)
vae_optimizer = optim.Adam(vae.parameters(), lr=CONFIG['vae_lr'])
gp_optimizer = optim.Adam(gp_params, lr=CONFIG['gp_lr'])
print(f"\n✅ Optimizers created:")
print(f"   VAE optimizer: Adam(lr={CONFIG['vae_lr']})")
print(f"   GP optimizer: Adam(lr={CONFIG['gp_lr']})")

VAE config: {'nf': 32, 'zdim': 256, 'vy': 0.002}

Loading pre-trained VAE...
✅ VAE loaded from ./out/vae_colab/20251224_171841/weights/weights.00099.pt
   Total VAE parameters: 553,304

Loading dataset...
✅ Data loaded:
   Training samples: 3868
   Validation samples: 484


AssertionError: Torch not compiled with CUDA enabled

## 11. Define Training Functions

These functions handle the complex GP-VAE training loop.

In [ ]:
def encode_Y(vae, train_queue):
    """Encode all training images to get latent codes"""
    vae.eval()

    with torch.no_grad():
        n = train_queue.dataset.Y.shape[0]
        Zm = Variable(torch.zeros(n, vae_cfg["zdim"]), requires_grad=False).cuda()
        Zs = Variable(torch.zeros(n, vae_cfg["zdim"]), requires_grad=False).cuda()

        for batch_i, data in enumerate(train_queue):
            y = data[0].cuda()
            idxs = data[-1].cuda()
            zm, zs = vae.encode(y)
            Zm[idxs], Zs[idxs] = zm.detach(), zs.detach()

    return Zm, Zs


def eval_step(vae, gp, vm, val_queue, Zm, Vt, Vv, Wv):
    """Enhanced evaluation with per-view metrics for Experiment #1"""
    rv = {}

    with torch.no_grad():
        _X = vm.x().data.cpu().numpy()
        _W = vm.v().data.cpu().numpy()
        covs = {"XX": np.dot(_X, _X.T), "WW": np.dot(_W, _W.T)}
        rv["vars"] = gp.get_vs().data.cpu().numpy()

        # Out-of-sample prediction
        vs = gp.get_vs()
        U, UBi, _ = gp.U_UBi_Shb([Vt], vs)
        Kiz = gp.solve(Zm, U, UBi, vs)
        Zo = vs[0] * Vv.mm(Vt.transpose(0, 1).mm(Kiz))

        mse_out = Variable(torch.zeros(Vv.shape[0], 1), requires_grad=False).cuda()
        mse_val = Variable(torch.zeros(Vv.shape[0], 1), requires_grad=False).cuda()

        for batch_i, data in enumerate(val_queue):
            idxs = data[-1].cuda()
            Yv = data[0].cuda()
            Zv = vae.encode(Yv)[0].detach()
            Yr = vae.decode(Zv)
            Yo = vae.decode(Zo[idxs])
            mse_out[idxs] = ((Yv - Yo) ** 2).view(Yv.shape[0], -1).mean(1)[:, None].detach()
            mse_val[idxs] = ((Yv - Yr) ** 2).view(Yv.shape[0], -1).mean(1)[:, None].detach()

            # Store examples for visualization
            if batch_i == 0:
                imgs = {}
                imgs["Yv"] = Yv[:24].data.cpu().numpy().transpose(0, 2, 3, 1)
                imgs["Yr"] = Yr[:24].data.cpu().numpy().transpose(0, 2, 3, 1)
                imgs["Yo"] = Yo[:24].data.cpu().numpy().transpose(0, 2, 3, 1)

        rv["mse_out"] = float(mse_out.data.mean().cpu())
        rv["mse_val"] = float(mse_val.data.mean().cpu())
        
        # NEW: Per-view metrics for Experiment #1
        unique_views = torch.unique(Wv).cpu().numpy()
        mse_val_per_view = {}
        mse_out_per_view = {}
        
        for view_idx in unique_views:
            view_mask = (Wv.cpu().numpy().flatten() == view_idx)
            mse_val_per_view[int(view_idx)] = float(mse_val.cpu().numpy()[view_mask].mean())
            mse_out_per_view[int(view_idx)] = float(mse_out.cpu().numpy()[view_mask].mean())
        
        rv['mse_val_per_view'] = mse_val_per_view
        rv['mse_out_per_view'] = mse_out_per_view

    return rv, imgs, covs


def backprop_and_update(vae, gp, vm, train_queue, Dt, Wt, Eps, Zb, Vbs, vbs, vae_optimizer, gp_optimizer):
    """Joint optimization of VAE and GP"""
    rv = {}

    vae_optimizer.zero_grad()
    gp_optimizer.zero_grad()
    vae.train()
    gp.train()
    vm.train()

    for batch_i, data in enumerate(train_queue):
        # Get batch data
        y = data[0].cuda()
        eps = Eps[data[-1]]
        _d = Dt[data[-1]]
        _w = Wt[data[-1]]
        _Zb = Zb[data[-1]]
        _Vbs = [Vbs[0][data[-1]]]

        # Forward through VAE
        zm, zs = vae.encode(y)
        z = zm + zs * eps
        yr = vae.decode(z)
        recon_term, mse = vae.nll(y, yr)

        # Forward through GP
        _Vs = [vm(_d, _w)]
        gp_nll_fo = gp.taylor_expansion(z, _Vs, _Zb, _Vbs, vbs) / vae.K

        # Penalization term
        pen_term = -0.5 * zs.sum(1)[:, None] / vae.K

        # Joint loss and backward
        loss = (recon_term + gp_nll_fo + pen_term).sum()
        loss.backward()

        # Accumulate metrics
        _n = train_queue.dataset.Y.shape[0]
        smartSum(rv, "mse", float(mse.data.sum().cpu()) / _n)
        smartSum(rv, "recon_term", float(recon_term.data.sum().cpu()) / _n)
        smartSum(rv, "pen_term", float(pen_term.data.sum().cpu()) / _n)

    # Update both optimizers
    vae_optimizer.step()
    gp_optimizer.step()

    return rv


print("✅ Training functions defined with per-view metrics")

✅ Training functions defined


## 12. Train GP-VAE Model 🚀

**This is joint optimization!** Both VAE and GP are updated together each iteration.

Training process per epoch:
1. Encode images to latent codes (VAE)
2. Compute GP prior likelihood on latents
3. Backpropagate through joint loss
4. Update VAE, GP, and Vmodel simultaneously

In [ ]:
import time
from IPython.display import clear_output

history = {}
start_time = time.time()

print(f"🚀 Starting GP-VAE Experiment #1 training for {CONFIG['epochs']} epochs...")
print("=" * 80)
print("Training mode: JOINT OPTIMIZATION (VAE + GP updated together)")
print(f"Experiment: Hard Held-Out Views")
print(f"  Training views: {CONFIG.get('train_view_indices', 'all')}")
print(f"  Validation views: {CONFIG.get('val_view_indices', 'all')}")
print("=" * 80)

for epoch in range(CONFIG['epochs']):
    epoch_start = time.time()

    # 1. Encode all training images
    Zm, Zs = encode_Y(vae, train_queue)

    # 2. Sample latent codes
    Eps = Variable(torch.randn(*Zs.shape), requires_grad=False).cuda()
    Z = Zm + Eps * Zs

    # 3. Compute variance matrices
    Vt = vm(Dt, Wt).detach()
    Vv = vm(Dv, Wv).detach()

    # 4. Evaluate on validation set (with per-view metrics)
    rv_eval, imgs, covs = eval_step(vae, gp, vm, val_queue, Zm, Vt, Vv, Wv)

    # 5. Compute GP Taylor expansion coefficients
    Zb, Vbs, vbs, gp_nll = gp.taylor_coeff(Z, [Vt])
    rv_eval["gp_nll"] = float(gp_nll.data.mean().cpu()) / vae.K

    # 6. Joint training step (VAE + GP)
    rv_back = backprop_and_update(
        vae, gp, vm, train_queue, Dt, Wt, Eps,
        Zb, Vbs, vbs, vae_optimizer, gp_optimizer
    )
    rv_back["loss"] = rv_back["recon_term"] + rv_eval["gp_nll"] + rv_back["pen_term"]

    # Store history
    smartAppendDict(history, rv_eval)
    smartAppendDict(history, rv_back)
    smartAppend(history, "vs", gp.get_vs().data.cpu().numpy())

    epoch_time = time.time() - epoch_start
    total_time = time.time() - start_time

    # 🔬 Compute diagnostic metrics
    train_val_gap = rv_back["mse"] - rv_eval["mse_val"]
    val_out_gap = rv_eval["mse_out"] - rv_eval["mse_val"]

    vs = gp.get_vs().data.cpu().numpy()
    variance_ratio = vs[0] / (vs[0] + vs[1])

    # Check if kernel has learnable lengthscale
    learned_lengthscale = None
    if hasattr(vm, 'view_kernel') and hasattr(vm.view_kernel, 'log_lengthscale'):
        learned_lengthscale = torch.exp(vm.view_kernel.log_lengthscale).item()

    # Print progress
    if epoch % 5 == 0 or epoch == CONFIG['epochs'] - 1:
        print(f"Epoch {epoch:4d}/{CONFIG['epochs']} | "
              f"MSE train: {rv_back['mse']:.6f} | "
              f"MSE val: {rv_eval['mse_val']:.6f} | "
              f"MSE out: {rv_eval['mse_out']:.6f} | "
              f"GP NLL: {rv_eval['gp_nll']:.4f} | "
              f"Gap(T-V): {train_val_gap:.6f} | "
              f"Gap(V-O): {val_out_gap:.6f} | "
              f"v₀/(v₀+v₁): {variance_ratio:.3f}" +
              (f" | ℓ: {learned_lengthscale:.3f}" if learned_lengthscale else "") +
              f" | Time: {epoch_time:.1f}s")
        
        # Print per-view breakdown (Experiment #1 specific)
        if CONFIG['view_split_mode'] == 'by_view' and epoch % 10 == 0:
            print("   Per-view MSE_out:")
            view_names = {0: "90L", 1: "60L", 2: "45L", 3: "30L", 4: "00F", 
                         5: "30R", 6: "45R", 7: "60R", 8: "90R"}
            for view_idx in sorted(rv_eval['mse_out_per_view'].keys()):
                mse = rv_eval['mse_out_per_view'][view_idx]
                view_name = view_names.get(view_idx, f"V{view_idx}")
                print(f"      {view_name}: {mse:.6f}")

    # Log to W&B
    if CONFIG['use_wandb']:
        log_dict = {
            "epoch": epoch,
            "mse_train": rv_back["mse"],
            "mse_val": rv_eval["mse_val"],
            "mse_out": rv_eval["mse_out"],
            "gp_nll": rv_eval["gp_nll"],
            "recon_term": rv_back["recon_term"],
            "pen_term": rv_back["pen_term"],
            "loss": rv_back["loss"],
            "vars": rv_eval["vars"],
            "time/epoch_seconds": epoch_time,
            # 🔬 Diagnostic metrics
            "diagnostics/gap_train_val": train_val_gap,
            "diagnostics/gap_val_out": val_out_gap,
            "diagnostics/variance_ratio": variance_ratio,
            "vars/v0_object": vs[0],
            "vars/v1_noise": vs[1],
        }

        # Add lengthscale if available
        if learned_lengthscale is not None:
            log_dict["kernel/lengthscale"] = learned_lengthscale
        
        # Add per-view metrics (Experiment #1 specific)
        if 'mse_val_per_view' in rv_eval:
            for view_idx, mse in rv_eval['mse_val_per_view'].items():
                view_names = {0: "90L", 1: "60L", 2: "45L", 3: "30L", 4: "00F",
                             5: "30R", 6: "45R", 7: "60R", 8: "90R"}
                view_name = view_names.get(view_idx, f"V{view_idx}")
                log_dict[f"mse_val_per_view/{view_name}"] = mse
        
        if 'mse_out_per_view' in rv_eval:
            for view_idx, mse in rv_eval['mse_out_per_view'].items():
                view_names = {0: "90L", 1: "60L", 2: "45L", 3: "30L", 4: "00F",
                             5: "30R", 6: "45R", 7: "60R", 8: "90R"}
                view_name = view_names.get(view_idx, f"V{view_idx}")
                log_dict[f"mse_out_per_view/{view_name}"] = mse

        wandb.log(log_dict)

    # Save checkpoint
    if epoch % CONFIG['epoch_cb'] == 0 or epoch == CONFIG['epochs'] - 1:
        logging.info(f"Epoch {epoch} - saving checkpoint")

        # Save VAE weights
        vae_file = os.path.join(wdir, f"vae_weights.{epoch:05d}.pt")
        torch.save(vae.state_dict(), vae_file)

        # Save GP weights
        gp_file = os.path.join(wdir, f"gp_weights.{epoch:05d}.pt")
        torch.save({
            'gp_state': gp.state_dict(),
            'vm_state': vm.state_dict(),
            'gp_params': gp_params.state_dict(),
        }, gp_file)

        # Save visualization
        ffile = os.path.join(fdir, f"plot.{epoch:05d}.png")
        callback_gppvae(epoch, history, covs, imgs, ffile)

        if CONFIG['use_wandb']:
            wandb.log({
                "reconstructions": wandb.Image(ffile),
                "covariances/XX": wandb.Image(ffile),
            })

        print(f"  ✓ Checkpoint saved at epoch {epoch}")

# At the end, enhanced summary with per-view breakdown
total_time = time.time() - start_time
print("\n" + "=" * 80)
print(f"✅ GP-VAE Experiment #1 training complete!")
print(f"   Total time: {total_time/60:.1f} minutes ({total_time/3600:.2f} hours)")
print(f"   Average time per epoch: {total_time/CONFIG['epochs']:.1f} seconds")
print(f"   Final training MSE: {rv_back['mse']:.6f}")
print(f"   Final validation MSE: {rv_eval['mse_val']:.6f}")
print(f"   Final out-of-sample MSE: {rv_eval['mse_out']:.6f}")
print(f"   Final GP NLL: {rv_eval['gp_nll']:.4f}")

print(f"\n🔬 Final Diagnostics:")
print(f"   Train-Val Gap: {train_val_gap:.6f} (lower = less overfitting)")
print(f"   Val-Out Gap: {val_out_gap:.6f} (lower = better GP interpolation)")
print(f"   Variance Ratio: {variance_ratio:.3f} (higher = more structure learned)")
if learned_lengthscale is not None:
    print(f"   Learned Lengthscale: {learned_lengthscale:.3f}")

# Experiment #1 specific: Per-view breakdown
if CONFIG['view_split_mode'] == 'by_view' and 'mse_out_per_view' in rv_eval:
    print(f"\n📊 Final Per-View MSE_out (Experiment #1):")
    view_names = {0: "90L (-90°)", 1: "60L (-60°)", 2: "45L (-45°)", 3: "30L (-30°)", 
                 4: "00F (0°)", 5: "30R (+30°)", 6: "45R (+45°)", 7: "60R (+60°)", 8: "90R (+90°)"}
    
    # Separate training and validation views
    train_view_indices = CONFIG.get('train_view_indices', [])
    val_view_indices = CONFIG.get('val_view_indices', [])
    
    print("   VALIDATION VIEWS (held-out, extreme angles):")
    extreme_mses = []
    for view_idx in sorted(rv_eval['mse_out_per_view'].keys()):
        if view_idx in val_view_indices:
            mse = rv_eval['mse_out_per_view'][view_idx]
            extreme_mses.append(mse)
            view_name = view_names.get(view_idx, f"V{view_idx}")
            print(f"      {view_name:15s}: {mse:.6f}")
    
    if extreme_mses:
        avg_extreme = np.mean(extreme_mses)
        print(f"\n   Average MSE on extreme angles: {avg_extreme:.6f}")
        print(f"   Overall MSE_out: {rv_eval['mse_out']:.6f}")

if CONFIG['use_wandb']:
    wandb.finish()
    print("\n🔗 View detailed results in W&B dashboard")

🚀 Starting GP-VAE training for 200 epochs...
Training mode: JOINT OPTIMIZATION (VAE + GP updated together)
Epoch    0/200 | MSE train: 0.003776 | MSE val: 0.004238 | MSE out: 0.068923 | GP NLL: 0.0020 | Gap(T-V): -0.000461 | Gap(V-O): 0.064686 | v₀/(v₀+v₁): 0.500 | Time: 5.9s
  ✓ Checkpoint saved at epoch 0
Epoch    5/200 | MSE train: 0.030871 | MSE val: 0.031063 | MSE out: 0.071995 | GP NLL: 0.0006 | Gap(T-V): -0.000192 | Gap(V-O): 0.040932 | v₀/(v₀+v₁): 0.497 | Time: 5.9s
Epoch   10/200 | MSE train: 0.014819 | MSE val: 0.015153 | MSE out: 0.056098 | GP NLL: 0.0008 | Gap(T-V): -0.000333 | Gap(V-O): 0.040946 | v₀/(v₀+v₁): 0.497 | Time: 6.0s
  ✓ Checkpoint saved at epoch 10
Epoch   15/200 | MSE train: 0.011634 | MSE val: 0.011731 | MSE out: 0.050627 | GP NLL: 0.0011 | Gap(T-V): -0.000097 | Gap(V-O): 0.038896 | v₀/(v₀+v₁): 0.496 | Time: 5.9s
Epoch   20/200 | MSE train: 0.008930 | MSE val: 0.009169 | MSE out: 0.045992 | GP NLL: 0.0007 | Gap(T-V): -0.000239 | Gap(V-O): 0.036823 | v₀/(v₀+v₁

diagnostics/gap_train_val,▁█▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
diagnostics/gap_val_out,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
diagnostics/variance_ratio,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇█
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█████
gp_nll,█▆▂▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,██▄▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse_out,█▇▆▇▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse_train,█▂▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse_val,▁▇█▅▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pen_term,▇▄▅▁▁▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████████
+4,...



🔗 View detailed results in W&B dashboard


## 13. Download Results

Download the trained model and visualizations to your computer:

In [ ]:
# Compress output folder
output_zip = '/content/gppvae_output.zip'
!zip -r {output_zip} {CONFIG['outdir']}

# Download
from google.colab import files
print("Preparing download...")
files.download(output_zip)
print("\n✅ Download started! Extract the zip on your local machine.")
print(f"\nContents include:")
print(f"  - Trained VAE weights (fine-tuned)")
print(f"  - GP + Vmodel weights")
print(f"  - Visualization plots")
print(f"  - Training logs")

## 14. Visualize Results

View the latest reconstruction and covariance plots: